In [1]:
import numpy as np
import opendssdirect as dss
import matplotlib.pyplot as plt
import pandas as pdk
import re
import sys
import time
import scipy.linalg as spla
from lib.compute_KCL_matrices import compute_KCL_matrices
from lib.basematricesY import basematrices as BM1
from lib.basematrices import basematrices as BM 
from lib.NR3_Y import NR3_timevarying as n1
from lib.NR3_timevarying import NR3_timevarying as n

Original NR3

In [2]:
t0 = time.time()
slackidx = 0
Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])

fn = 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'
#fn = '06node_threephase_unbalanced.dss'
#fn = 'IEEE_13Node_Modified_01.dss'
fn = '02node_threephase_unbalanced_new.dss'
#fn = '06node_threephase_meshed_unbalanced.dss'
times = [-1]
der = [0]
capacitance =[0]

X, g_SB, b_SB, G_KVL, b_KVL, H, g, b = BM(fn, slackidx, Vslack, None, None)

   
dss.run_command('Redirect ' + fn)
nnode = len(dss.Circuit.AllBusNames())
nline = len(dss.Lines.AllNames())

VNR02 = np.zeros((len(times), 3, nnode), dtype = "complex")
INR01=  np.zeros((len(times), 3, nline), dtype = "complex")
STXNR01 = np.zeros((len(times), 3, nline), dtype = "complex")
SRXNR01 = np.zeros((len(times), 3, nline), dtype = "complex")
iNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")
sNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")

t3 = time.time()

for _ in range(1):
    VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = n(fn, X, g_SB, b_SB, G_KVL, b_KVL, H, g, b, None, None, 0, 0, 0)    
VNR02[0, :, :] = np.reshape(VNR, (3, nnode))

t2 = time.time()

Iteration number 0.000000
Iteration number 1.000000
Iteration number 2.000000
VNR
[[ 1.        +0.00000000e+00j  0.99969566+7.20406127e-04j
   0.9994932 +1.17248712e-03j]
 [-0.5       -8.66025404e-01j -0.49933891-8.66488819e-01j
  -0.49895981-8.66509948e-01j]
 [-0.5       +8.66025404e-01j -0.50072365+8.65528039e-01j
  -0.50105707+8.65335819e-01j]]
INR:
[[ 0.22511018-2.98016442e-02j  0.10005057+1.17367485e-04j]
 [-0.15129726-1.72505916e-01j -0.06671584-1.05840072e-01j]
 [-0.0443896 +1.76508443e-01j -0.05011251+8.65453293e-02j]]
STXNR:
[[0.22511018+2.98016442e-02j 0.1000202 -4.52547236e-05j]
 [0.22504313+4.47743114e-02j 0.12502306+4.95846653e-03j]
 [0.17505559+4.98117034e-02j 0.09999993-3.84879379e-05j]]
SRXNR:
[[0.2250202 +0.02995475j 0.1       +0.j        ]
 [0.22502306+0.04495847j 0.125     +0.005j     ]
 [0.17499993+0.04996151j 0.1       +0.j        ]]
iNR
[[ 0.        +0.j          0.12505961-0.02991901j  0.10007404-0.01989275j]
 [ 0.        +0.j         -0.08458141-0.06666584j -0.0

In [3]:
print('nr3',  t2-t3)
print( 'basematrices', t3 - t0)

nr3 0.047870635986328125
basematrices 0.024291276931762695


In [4]:
print(VNR)

[[ 1.        +0.00000000e+00j  0.99969566+7.20406127e-04j
   0.9994932 +1.17248712e-03j]
 [-0.5       -8.66025404e-01j -0.49933891-8.66488819e-01j
  -0.49895981-8.66509948e-01j]
 [-0.5       +8.66025404e-01j -0.50072365+8.65528039e-01j
  -0.50105707+8.65335819e-01j]]


Adjusted NR3

In [5]:
t0 = time.time()
slackidx = 0
Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])

fn = 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'
#fn = '06node_threephase_unbalanced.dss'
#fn = 'IEEE_13Node_Modified_01.dss'
fn = '02node_threephase_unbalanced_new.dss'
#fn = '06node_threephase_meshed_unbalanced.dss'
times = [-1]
der = [0]
capacitance =[0]

X, g_SB, b_SB, H, g, b = BM1(fn, slackidx, Vslack, None, None)
 
dss.run_command('Redirect ' + fn)
nnode = len(dss.Circuit.AllBusNames())
nline = len(dss.Lines.AllNames())

VNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")
INR01=  np.zeros((len(times), 3, nline), dtype = "complex")
STXNR01 = np.zeros((len(times), 3, nline), dtype = "complex")
SRXNR01 = np.zeros((len(times), 3, nline), dtype = "complex")
iNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")
sNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")

t3 = time.time()
for _ in range(1):
    # Run NR3 with variations in time, DER, and capacitance
# VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
    VNR = n1(fn, X, g_SB, b_SB, H, g, b, None, None, 0, 0, 0)    
VNR01[0, :, :] = np.reshape(VNR, (3, nnode))

t2 = time.time()

Iteration number 0.000000
Iteration number 1.000000
Iteration number 2.000000
Iteration number 3.000000
VNR
[[ 1.        +0.j          0.98448377+0.00416667j  0.96896754+0.00833333j]
 [-0.5       -0.8660254j  -0.48863345-0.85467129j -0.47726689-0.84331717j]
 [-0.5       +0.8660254j  -0.49585032+0.85050462j -0.49170065+0.83498384j]]


In [6]:
print(VNR)

[[ 1.        +0.j          0.98448377+0.00416667j  0.96896754+0.00833333j]
 [-0.5       -0.8660254j  -0.48863345-0.85467129j -0.47726689-0.84331717j]
 [-0.5       +0.8660254j  -0.49585032+0.85050462j -0.49170065+0.83498384j]]


In [7]:
print('nr3',  t2-t3)
print( 'basematrices', t3 - t0)

nr3 0.017260313034057617
basematrices 0.030970096588134766


OpenDSS Solution

In [8]:
dss.run_command('Redirect compare_opendss_05node_threephase_unbalanced_oscillation_03.dss')
#dss.run_command('Redirect 06node_threephase_unbalanced.dss') 
#dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
#dss.run_command('Redirect 06node_threephase_meshed_unbalanced.dss')
dss.run_command('Redirect 02node_threephase_unbalanced_new.dss')

VDSS0 = np.zeros((len(times), 3, nnode), dtype = "complex")
IDSS0 =  np.zeros((len(times), 3, nline), dtype="complex")
SRXDSS0 = np.zeros((len(times), 3, nline), dtype="complex")
STXDSS0 = np.zeros((len(times), 3, nline), dtype="complex")
load_arr = np.zeros((len(times), 3, nnode), dtype='complex')

kW_list = np.array([])
kvar_list = np.array([])

dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[0])
Vbase = dss.Bus.kVBase() * 1000
Sbase = 1000000.0
Ibase = Sbase/Vbase
Zbase = Vbase/Ibase

# Set slack bus (sourcebus) voltage reference in p.u.
SlackBusVoltage = 1.000
dss.Vsources.PU(SlackBusVoltage)

dss.Solution.Convergence(0.000000000001)
t4 = time.time()
# Solve power flow with OpenDSS file
for _ in range(1):
    dss.Solution.Solve()
    if not dss.Solution.Converged:
        print('Initial Solution Not Converged. Check Model for Convergence')
    else:
        print('Initial Model Converged. Proceeding to Next Step.')
        #Doing this solve command is required for GridPV, that is why the monitors
        #go under a reset process
        dss.Monitors.ResetAll()

        # set solution Params
        #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
        dss.Solution.Mode(1)
        dss.Solution.Number(1)
        dss.Solution.StepSize(1)
        dss.Solution.ControlMode(-1)
        dss.Solution.MaxControlIterations(1000000)
        dss.Solution.MaxIterations(30000)
        # Easy process to get all names and count of loads, a trick to avoid
        # some more lines of code
        TotalLoads=dss.Loads.Count()
        AllLoadNames=dss.Loads.AllNames()
        print('OpenDSS Model Compliation Done.')
        print('Iterations: ', dss.Solution.Iterations())
        print('Tolerance: ', dss.Solution.Convergence())

    print('')

t5=time.time()
print(dss.Solution.Converged())

# Print number of buses, and bus names
print(len(dss.Circuit.AllBusNames()))
print(dss.Circuit.AllBusNames())

# Print number of loads, and load names
print(len(dss.Loads.AllNames()))
print(dss.Loads.AllNames())

print('')

VDSS = np.zeros((3,len(dss.Circuit.AllBusNames())),dtype='complex')

for k1 in range(len(dss.Circuit.AllBusNames())):

    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])

#     print(dss.Circuit.AllBusNames()[k1])
#     print(dss.Bus.Nodes())

#     print('puVOTLAGES - LN CARTESIAN')
#     print(dss.Bus.PuVoltage())

    ph = np.asarray(dss.Bus.Nodes(),dtype='int')-1

    Vtemp = np.asarray(dss.Bus.PuVoltage())

    Vtemp = Vtemp[0:nnode*2-1:2] + 1j*Vtemp[1:nnode*2:2]


#     print(np.asarray(dss.Bus.Nodes(),'int'))

    VDSS[ph,k1] = Vtemp    

#     VDSS[np.asarray(dss.Bus.Nodes(),'int'),k1] = np.array(dss.Bus.PuVoltage()[0:5:2] + 1j*dss.Bus.PuVoltage()[1:6:2])


#     VDSS[dss.Bus.Nodes()-1,k1] = dss.Bus.PuVoltage()[0:2:5]
#     for k2 in range(len(dss.Bus.Nodes())):
#         VDSS[int(dss.Bus.Nodes()[k2])-1,k1] = dss.Bus.PuVoltage()[2*k2] + 1j*dss.Bus.PuVoltage()[2*k2+1]

    print('VDSS\n', np.round(VDSS,decimals=6))
VDSS0[0, :, :] = VDSS

#     IDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

#     for k1 in range(len(dss.Lines.AllNames())):
#         dss.Lines.Name(dss.Lines.AllNames()[k1])
#     #     print(dss.Lines.AllNames()[k1])
#         ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
#         Imn = np.asarray(dss.CktElement.Currents())/Ibase
#     #     print(Imn)
#         Imn = Imn[0:int(len(Imn)/2)]
#     #     print(Imn)
#         Imn = Imn[0:(len(ph)*2)-1:2] + 1j*Imn[1:len(ph)*2:2]
#     #     print(Imn)
#         IDSS[ph,k1] = Imn
#     #     print('')

#     print('IDSS\n', np.round(IDSS,decimals=6))

#     STXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')
#     SRXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

#     for k1 in range(len(dss.Lines.AllNames())):
#         dss.Lines.Name(dss.Lines.AllNames()[k1])
#         ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
#         Sk = np.asarray(dss.CktElement.Powers())/(Sbase/1000)

#         STXtemp = Sk[0:int(len(Sk)/2)]
#         SRXtemp = Sk[int(len(Sk)/2):]

#         STXtemp = STXtemp[0:len(ph)*2-1:2] + 1j*STXtemp[1:len(ph)*2:2]
#         SRXtemp = -(SRXtemp[0:len(ph)*2-1:2] + 1j*SRXtemp[1:len(ph)*2:2])

#         STXDSS[ph,k1] = STXtemp
#         SRXDSS[ph,k1] = SRXtemp

#VDSS0[i, :, :] = VDSS
#     IDSS0[i, :, :] = IDSS
#     SRXDSS0[i, :, :] = SRXDSS
#     STXDSS0[i, :, :] = STXDSS


#     print('STXDSS\n', np.round(STXDSS,decimals=6))
#     print('SRXDSS\n', np.round(SRXDSS,decimals=6))

#     print('|VDSS|\n', np.round(np.abs(VDSS),decimals=6))
#     print('<VDSS\n', np.round(180/np.pi*np.angle(VDSS),decimals=6))
#     print('D<VDSS\n', 180/np.pi*np.angle(VDSS) - 180/np.pi*np.angle(VDSS[:,[0]]))


Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.
Iterations:  2
Tolerance:  1e-12

True
3
['sourcebus', 'a01', 'a02']
6
['load_a01_a_01', 'load_a01_b_01', 'load_a01_c_01', 'load_a02_a_01', 'load_a02_b_01', 'load_a02_c_01']

VDSS
 [[ 0.999999-3.00000e-06j  0.      +0.00000e+00j  0.      +0.00000e+00j]
 [-0.500002-8.66023e-01j  0.      +0.00000e+00j  0.      +0.00000e+00j]
 [-0.499997+8.66025e-01j  0.      +0.00000e+00j  0.      +0.00000e+00j]]
VDSS
 [[ 0.999999-3.00000e-06j  0.999694+7.17000e-04j  0.      +0.00000e+00j]
 [-0.500002-8.66023e-01j -0.499341-8.66486e-01j  0.      +0.00000e+00j]
 [-0.499997+8.66025e-01j -0.500721+8.65528e-01j  0.      +0.00000e+00j]]
VDSS
 [[ 0.999999-3.00000e-06j  0.999694+7.17000e-04j  0.999492+1.16900e-03j]
 [-0.500002-8.66023e-01j -0.499341-8.66486e-01j -0.498962-8.66507e-01j]
 [-0.499997+8.66025e-01j -0.500721+8.65528e-01j -0.501054+8.65336e-01j]]


In [9]:
t5-t4

0.0004551410675048828

Comparing results between three solutions - DSS, 01 (vectorized), 02 (nonvectorized)

In [10]:
for i in range(len(times)):
    print("Timestep %d \n" % i)
    print('Complex Voltage difference')
    #print(np.max(np.abs(VNR01[i] - VNR02[i])))
    print(np.max(np.abs(VDSS0[i] - VNR01[i])))  #dss vs adjusted
    print(np.max(np.abs(VDSS0[i] - VNR02[i]))) #dss vs original
    print(np.max(np.abs(VNR01[i] - VNR02[i]))) #original vs adjusted

#     print('Complex Current difference')
#     print(np.max(np.abs(IDSS0[i] - INR01[i])))
    
#     print('Complex TX Power difference')
#     print(np.max(np.abs(STXDSS0[i] - STXNR01[i])))
    
#     print('Complex RX Power difference')
#     print(np.max(np.abs(SRXDSS0[i] - SRXNR01[i])))
    
#     print('Voltage Magnitude difference')
#     print(np.max(np.abs(np.abs(VDSS0[i]) - np.abs(VNR01[i]))))
    
#     print('Voltage Angle difference')
#     print(np.max(np.abs(180/np.pi*np.angle(VDSS0[i]) - 180/np.pi*np.angle(VNR01[i]))))
#     print("\n")

Timestep 0 

Complex Voltage difference
0.03176062897825715
3.6044738542345292e-06
0.031761378925861825


Load comparison between non-vectorized and OpenDSS

In [ ]:
for i in range(len(times)):
    print("Timestep %d \n Magnitude of the Difference: \n" % i)
    print(np.abs(spah[i] - load_arr[i]))
    print("Max difference: \n ")
    print(np.max(np.abs(spah[i] - load_arr[i])))
    print("\n")

In [ ]:
dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
dss.run_command('Redirect 02node_threephase_unbalanced_new.dss')
for i in range(len(dss.Loads.AllNames())):
    dss.Loads.Name(dss.Loads.AllNames()[i])
    print(dss.Loads.kW())
    print(dss.Loads.kvar())
    print(dss.CktElement.Powers())

---

In [ ]:
t = -1
def load_order_f():
    load_order = {}
    for n in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[n])
        pattern =  r"(\w+)\."
        load_bus = re.findall(pattern, dss.CktElement.BusNames()[0])
        if load_bus[0] not in load_order:
            load_order[load_bus[0]] = 1
        elif load_bus[0] in load_order:
            load_order[load_bus[0]] += 1
    return load_order
load_order_list = load_order_f()

def load_values():
    load_ph_arr = np.zeros((nnode, max(load_order_list.values()), 3))
    load_kw_arr_ph = np.zeros((3, nnode))
    load_kvar_arr_ph = np.zeros((3, nnode))
    if t == -1:
        var = 1
    else:
        var = (1 + 0.1*np.sin(2*np.pi*0.01*t))
    for load in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[load])
        load_data = dss.CktElement.BusNames()[0].split('.')
        idxbs = dss.Circuit.AllBusNames().index(load_data[0])
        for ph in range(1, len(load_data)):
            load_kw_arr_ph[int(load_data[ph]) - 1, idxbs] += dss.Loads.kW() * 1e3 / Sbase / (len(load_data) - 1)
            load_kvar_arr_ph[int(load_data[ph]) - 1, idxbs] += dss.Loads.kvar() * 1e3 / Sbase / (len(load_data) - 1) 
    return load_kw_arr_ph, load_kvar_arr_ph
load_kw_arr_ph, load_kvar_arr_ph = load_values()
print(load_kw_arr_ph)
print(load_kvar_arr_ph)

In [ ]:

            idxbs = dss.Circuit.AllBusNames().index(cap_data[0])
            for ph in range(1, len(cap_data)):
                caparr[int(cap_data[ph]) - 1, idxbs] += dss.Capacitors.kvar() * 1e3 / Sbase / (len(cap_data) - 1)
        return caparr

In [ ]:
[[0.         0.         0.10384609 0.10384609 0.10384609 0.15576913
  0.        ]
 [0.         0.         0.10384609 0.10384609 0.         0.10384609
  0.05192304]
 [0.         0.         0.10384609 0.10384609 0.10384609 0.
  0.        ]]
[[0.         0.         0.02596152 0.02596152 0.02596152 0.05192304
  0.        ]
 [0.         0.         0.02596152 0.02596152 0.         0.02596152
  0.02596152]
 [0.         0.         0.02596152 0.02596152 0.02596152 0.
  0.        ]]